**Objective** -- To create ladder structure of all teams in Australian Football League from raw data.  
Ladder is team standings based on number of games won, draw and lost.  
http://www.afl.com.au/ladder

**Rules** --
1. Each game won earns 4 points, drawn 2 and lost 0.  
2. Games are won by scoring more points in the game. Each goal scored is 6 points and behind is 1.  
3. If teams are tied in total points on the ladder, percentage p = total points scored/total points scored against is used. Higher percent moves the team higher in the ranking.  


**Tools used** --
Using pandas mainly to read, formar, structure and calculate i.e. data munging to create ladders

In [13]:
# Load libraries needed.
import pandas as pd
import numpy as np
# Display full content of column instead of truncated
pd.set_option('display.max_colwidth', -1)

**Load Data** --  
Raw data in text format is available @ https://afltables.com/afl/stats/biglists/bg3.txt  
Download the file.

In [14]:
file = 'bg3_updated.txt'
cols = ['game_num','date','round','home_team','home_score','away_team','away_score','ground']  

# field seperator is multiple spaces. Strings such as team names and ground also have single spaces. Game number field has .
# Using regular expression to parse data

df = pd.read_csv(file,skiprows=2,sep='[. ] +',header=None,names = cols, engine = 'python',parse_dates=['date'])
df.head(10)

,game_num,date,round,home_team,home_score,away_team,away_score,ground
0,1,1897-05-08,R1,Fitzroy,6.13.49,Carlton,2.4.16,Brunswick St
1,2,1897-05-08,R1,Collingwood,5.11.41,St Kilda,2.4.16,Victoria Park
2,3,1897-05-08,R1,Geelong,3.6.24,Essendon,7.5.47,Corio Oval
3,4,1897-05-08,R1,South Melbourne,3.9.27,Melbourne,6.8.44,Lake Oval
4,5,1897-05-15,R2,South Melbourne,6.4.40,Carlton,5.6.36,Lake Oval
5,6,1897-05-15,R2,Essendon,4.6.30,Collingwood,8.2.50,East Melbourne
6,7,1897-05-15,R2,St Kilda,3.8.26,Fitzroy,10.6.66,Junction Oval
7,8,1897-05-15,R2,Melbourne,9.10.64,Geelong,3.1.19,M.C.G.
8,9,1897-05-22,R3,Collingwood,6.5.41,Geelong,5.7.37,Victoria Park
9,10,1897-05-22,R3,Fitzroy,5.9.39,Melbourne,7.8.50,Brunswick St


In [15]:
# 15,407 game records.
df.shape

(15407, 8)

In [17]:
# records from 1897 till 2018
df.date.dt.year.min(), df.date.dt.year.max()

(1897, 2018)

In [18]:
df['round'].unique()

array(['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11',
       'R12', 'R13', 'R14', 'SF', 'R15', 'R16', 'R17', 'GF', 'PF', 'R18',
       'R19', 'R20', 'R21', 'R22', 'QF', 'EF', 'R23', 'R24'], dtype=object)

R* are round robin league stage, QF - Quarter final, SF - Semifinal, PF- Prefinal, EF - Final rounds. 
Team standings are accumulative. i.e. standing in R3 includes results from R1 and R2

In [72]:
def create_ladder(yr=2018, rnd = 'R23'):
    '''Year of play from 1897 to 2018.
    rnd is round robin league R1 to R23. 
    After round robin the rules have changed over course of history. And they depend on few matches. To keep it simple only
    round robin level standing are calculated.
    '''
    'Get the global dataframe df created above.'
    global df
    rr_list = ['R0','R1','R2','R3','R4','R5','R6','R7','R8','R9','R10','R11','R12','R13','R14','R15','R16',
              'R17','R18','R19','R20','R21','R22','R23']
    
    'Filter by year of play'
    df_2016 = df[df.date.dt.year == yr]
    
    'Filter by round robin level. R3 includes plays from R1 and R2 as well as R3.'
    'Since R12 is less than R2 by string compare, using the static list above and get the index values'
    'This will return all Rx before rnd input'
    
    df_2016 = df_2016[(df_2016['round'].isin(rr_list[:rr_list.index(rnd)+1]))]
    
    'Get home teams scores'
    df_home = df_2016.home_score.str.split('.',expand = True).rename(columns = {0:'home_goals',
                                                                                1:'home_behinds',
                                                                                2:'home_points'})
    'Get away team scores. '
    df_away = df_2016.away_score.str.split('.',expand = True).rename(columns = {0:'away_goals',
                                                                                1:'away_behinds',
                                                                                2:'away_points'})
    df_home = df_home.astype('int')
    df_away = df_away.astype('int')
    
    merged_df = pd.concat([df_2016,df_home,df_away],axis=1)
    merged_df['played'] = 1
    home_team_df =  merged_df[['home_team','home_points','away_points','played']].copy()
    home_team_df['match_points'] = np.where(home_team_df.home_points > home_team_df.away_points, 4,
                                            np.where(home_team_df.home_points == home_team_df.away_points,2,0))
    away_team_df = merged_df[['away_team','away_points','home_points','played']].copy()
    away_team_df['match_points'] = np.where(away_team_df.home_points > away_team_df.away_points, 0,
                                            np.where(away_team_df.home_points == away_team_df.away_points,2,4))
    home_team_df.rename(columns = {'home_team':'Team','home_points':'F','away_points':'A'},inplace=True)
    away_team_df.rename(columns = {'away_team':'Team','home_points':'A','away_points':'F'},inplace=True)
    combined_df = pd.concat([home_team_df,away_team_df])
    combined_df = combined_df.groupby(by=['Team']).agg('sum').reset_index()
    combined_df = pd.concat([home_team_df,away_team_df])
    combined_df = combined_df.groupby(by=['Team']).agg('sum').reset_index()
    combined_df['percent'] = combined_df['F']/combined_df['A']*100
    combined_df.sort_values(by=['match_points','percent'], ascending = [False,False],inplace = True)
    combined_df['pos'] = np.arange(1,len(combined_df)+1)
    combined_df.reset_index(inplace=True,drop=True)
    return combined_df
    

In [74]:
df1 = create_ladder(yr=2013,rnd = 'R12')
print(df1)

                Team     F     A  played  match_points     percent  pos
0   Hawthorn          1307  897   11      40            145.707915  1  
1   Geelong           1291  997   11      40            129.488465  2  
2   Essendon          1267  963   12      36            131.568017  3  
3   Sydney            1144  848   11      34            134.905660  4  
4   Fremantle         990   770   11      34            128.571429  5  
5   Collingwood       1152  1062  12      32            108.474576  6  
6   Richmond          1081  946   11      28            114.270613  7  
7   Carlton           1211  1036  12      24            116.891892  8  
8   West Coast        1135  987   11      24            114.994934  9  
9   Port Adelaide     1071  935   11      24            114.545455  10 
10  Adelaide          1108  1070  12      20            103.551402  11 
11  Gold Coast        1031  1114  12      20            92.549372   12 
12  North Melbourne   1148  1033  11      16            111.1326